In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import tensorflow_datasets as tfds
import os

This tutorial provides an example of how to use tf.data.TextLineDataset to load examples from text files. TextLineDataset is designed to create a dataset from a text file, in which each example is a line of text from the original file. This is potentially useful for any text data that is primarily line-based (for example, poetry or error logs).

In this tutorial, we'll use three different English translations of the same work, Homer's Illiad, and train a model to identify the translator given a single line of text.

In [2]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
    text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)
    
parent_dir = os.path.dirname(text_dir)

parent_dir

'/Users/justintennenbaum/.keras/datasets'

# Load text into datasets
Iterate through the files, loading each one into its own dataset.

Each example needs to be individually labeled, so use tf.data.Dataset.map to apply a labeler function to each one. This will iterate over every example in the dataset, returning (example, label) pairs.

In [3]:
def labeler(example, index):
    return example, tf.cast(index, tf.int64)

labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
    lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
    labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
    labeled_data_sets.append(labeled_dataset)

In [4]:
#combine these labeled documents into a single dataset and shuffle it
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [5]:
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
    all_labeled_data = all_labeled_data.concatenate(labeled_dataset)
    
all_labeled_data = all_labeled_data.shuffle(BUFFER_SIZE,
                                           reshuffle_each_iteration=False)

You can use tf.data.Dataset.take and print to see what the (example, label) pairs look like. The numpy property shows each Tensor's value.

In [6]:
for ex in all_labeled_data.take(5):
    print(ex,'\n')

(<tf.Tensor: shape=(), dtype=string, numpy=b'But since on man ten thousand forms of death'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>) 

(<tf.Tensor: shape=(), dtype=string, numpy=b'The Gods in council. She to every part'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>) 

(<tf.Tensor: shape=(), dtype=string, numpy=b'spoke fiercely to him and sent him roughly away. "Old man," said he,'>, <tf.Tensor: shape=(), dtype=int64, numpy=2>) 

(<tf.Tensor: shape=(), dtype=string, numpy=b'To godlike honors in all Trojan hearts,'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>) 

(<tf.Tensor: shape=(), dtype=string, numpy=b'Of gold and brass, when we divide the spoil,'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>) 



# Encode text lines as numbers
Machine learning models work on numbers, not words, so the string values need to be converted into lists of numbers. To do that, map each unique word to a unique integer.

# Build vocabulary
First, build a vocabulary by tokenizing the text into a collection of individual unique words. There are a few ways to do this in both TensorFlow and Python. For this tutorial:

1) Iterate over each example's numpy value.

2) Use tfds.features.text.Tokenizer to split it into tokens.

3) Collect these tokens into a Python set, to remove duplicates.

4) Get the size of the vocabulary for later use.

In [7]:
tokenizer = tfds.features.text.Tokenizer()
vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
    some_tokens = tokenizer.tokenize(text_tensor.numpy())
    vocabulary_set.update(some_tokens)
    
vocab_size = len(vocabulary_set)
vocab_size

17178

# Encode examples
Create an encoder by passing the vocabulary_set to tfds.features.text.TokenTextEncoder. The encoder's encode method takes in a string of text and returns a list of integers.

In [8]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [9]:
#you can try this on a single line to see what the output looks like
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)

b'But since on man ten thousand forms of death'


In [10]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

[15211, 9592, 1375, 11598, 7192, 13705, 4636, 5046, 2754]


Now run the encoder on the dataset by wrapping it in tf.py_function and passing that to the dataset's map method.

In [11]:
def encode(text_tensor, label):
    encoded_text = encoder.encode(text_tensor.numpy())
    return encoded_text, label

You want to use Dataset.map to apply this function to each element of the dataset. Dataset.map runs in graph mode.

- Graph tensors do not have a value.

- In graph mode you can only use TensorFlow Ops and functions.


So you can't .map this function directly: You need to wrap it in a tf.py_function. The tf.py_function will pass regular tensors (with a value and a .numpy() method to access it), to the wrapped python function.

In [12]:
def encode_map_fn(text, label):
  # py_func doesn't set the shape of the returned tensors.
    encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int64))

  # `tf.data.Datasets` work best if all components have a shape set
  #  so set the shapes manually: 
    encoded_text.set_shape([None])
    label.set_shape([])

    return encoded_text, label


all_encoded_data = all_labeled_data.map(encode_map_fn)

# Split the dataset into test and train batches
Use tf.data.Dataset.take and tf.data.Dataset.skip to create a small test dataset and a larger training set.

Before being passed into the model, the datasets need to be batched. Typically, the examples inside of a batch need to be the same size and shape. But, the examples in these datasets are not all the same size — each line of text had a different number of words. So use tf.data.Dataset.padded_batch (instead of batch) to pad the examples to the same size.

In [15]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE,padded_shapes = ([None],()))

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE,padded_shapes = ([None],()))

Now, test_data and train_data are not collections of (example, label) pairs, but collections of batches. Each batch is a pair of (many examples, many labels) represented as arrays.

To illustrate:

In [16]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0], sample_labels[0]

(<tf.Tensor: shape=(18,), dtype=int64, numpy=
 array([15211,  9592,  1375, 11598,  7192, 13705,  4636,  5046,  2754,
            0,     0,     0,     0,     0,     0,     0,     0,     0])>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>)

Since we have introduced a new token encoding (the zero used for padding), the vocabulary size has increased by one.

In [17]:
vocab_size += 1

# Build the model

In [18]:
model = tf.keras.Sequential()

#The first layer converts integer representations to dense vector embeddings
model.add(tf.keras.layers.Embedding(vocab_size,64))

The next layer is a Long Short-Term Memory layer, which lets the model understand words in their context with other words. A bidirectional wrapper on the LSTM helps it to learn about the datapoints in relationship to the datapoints that came before it and after it.

In [19]:
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

Finally we'll have a series of one or more densely connected layers, with the last one being the output layer. The output layer produces a probability for all the labels. The one with the highest probability is the models prediction of an example's label.



In [20]:
#one or more dense layers
#edit the lit in the 'for' line to experiment with layer sizes
for units in [64,64]:
    model.add(tf.keras.layers.Dense(units, activation='relu'))
    
#output layer
model.add(tf.keras.layers.Dense(3))

Finally, compile the model. For a softmax categorization model, use sparse_categorical_crossentropy as the loss function. You can try other optimizers, but adam is very common.



In [22]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])

# Train the model
This model running on this data produces decent results (about 83%).

In [23]:
model.fit(train_data, epochs=3, validation_data=test_data)

Epoch 1/3
697/697 [==============================] - 43s 62ms/step - loss: 0.5335 - accuracy: 0.7305 - val_loss: 0.3925 - val_accuracy: 0.8196
Epoch 2/3
697/697 [==============================] - 34s 49ms/step - loss: 0.3084 - accuracy: 0.8644 - val_loss: 0.3478 - val_accuracy: 0.8368
Epoch 3/3
697/697 [==============================] - 34s 49ms/step - loss: 0.2310 - accuracy: 0.8991 - val_loss: 0.3631 - val_accuracy: 0.8422


In [24]:
eval_loss, eval_acc = model.evaluate(test_data)
print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))

     79/Unknown - 4s 48ms/step - loss: 0.3631 - accuracy: 0.8422
Eval loss: 0.363, Eval accuracy: 0.842
